In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="google/flan-t5-large")

Device set to use cpu


In [ ]:
%pip install flask transformers torch


Note: you may need to restart the kernel to use updated packages.


: 

In [1]:
from flask import Flask, request, jsonify
from transformers import pipeline
import torch
import time
import threading

# Check for GPU availability
device = 0 if torch.cuda.is_available() else -1

# Load the translation model (Supports instruction prompts like "translate English to French")
translator = pipeline("text2text-generation", model="google/flan-t5-large", device=device)

# Initialize Flask
app = Flask(__name__)

@app.route('/')
def home():
    return "🟢 Translation API is running!"

@app.route('/translate', methods=['POST'])
def translate_text():
    try:
        # Get request data
        data = request.get_json()

        # Validate input text
        text = data.get('text', '').strip()
        if not text:
            return jsonify({'error': 'Text must be a non-empty string'}), 400

        # Get custom prompt info
        source_lang = data.get('source_lang', 'English').strip()
        target_lang = data.get('target_lang', 'French').strip()

        if not source_lang or not target_lang:
            return jsonify({'error': 'Both source_lang and target_lang must be provided'}), 400

        # Create prompt dynamically
        prompt = f"translate {source_lang} to {target_lang}: {text}"

        # Perform translation
        translated_text = translator(prompt, max_length=256, do_sample=False)

        return jsonify({
            'translation': translated_text[0]['generated_text'],
            'source_lang': source_lang,
            'target_lang': target_lang
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run Flask app in a separate thread for integration
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

time.sleep(2)  # Allow server time to start


Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.17:5000
Press CTRL+C to quit


127.0.0.1 - - [10/Apr/2025 18:57:16] "GET / HTTP/1.1" 200 -
192.168.1.17 - - [10/Apr/2025 18:58:29] "GET / HTTP/1.1" 200 -
192.168.1.17 - - [10/Apr/2025 18:58:29] "GET /favicon.ico HTTP/1.1" 404 -
